### **High Frequency Trading Strategies Design using ML and DL**

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 13 22:40:00 2022

@author: Bradley

HFT: Machine Learning Techniques on Full Orderbook Tick Data
"""

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from datetime import datetime
import csv

plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (8,5) #提前设置图片形状大小

%config InlineBackend.figure_format = 'svg'

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')  # 忽略一些warnings

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
pd.set_option('expand_frame_repr', False)
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.width', 180)

### **Feature Engineering**

In [19]:
# read the data
def order_book(month,day):
    datapath = './data/order_book_3_2014' + '_' + str(month) + '_' + str(day) + '.csv'
    order = pd.read_csv(datapath,sep=',') #隔一个取一个
    bid_price_1 = np.array(order['Bid'][1::4].astype("float64"))/100 #bid1
    bid_price_2 = np.array(order['Bid'][2::4].astype("float64"))/100 #bid2
    bid_price_3 = np.array(order['Bid'][3::4].astype("float64"))/100 #bid3
    timestamp = np.array(order['Bid_Quantity'][0::4]) #每个tick data的时间
    bid_quantity_1 = np.array(order['Bid_Quantity'][1::4].astype("float64"))/100 #bidqty1
    bid_quantity_2 = np.array(order['Bid_Quantity'][2::4].astype("float64"))/100 #bidqty2
    bid_quantity_3 = np.array(order['Bid_Quantity'][3::4].astype("float64"))/100 #bidqty3
    ask_price_1 = np.array(order['Ask'][1::4].astype("float64"))/100 #ask1
    ask_price_2 = np.array(order['Ask'][2::4].astype("float64"))/100 #ask2
    ask_price_3 = np.array(order['Ask'][3::4].astype("float64"))/100 #ask3
    ask_quantity_1 = np.array(order['Ask_Quantity'][1::4].astype("float64"))/100 #askqty1
    ask_quantity_2 = np.array(order['Ask_Quantity'][2::4].astype("float64"))/100 #askqty1
    ask_quantity_3 = np.array(order['Ask_Quantity'][3::4].astype("float64"))/100 #askqty1
    
    # 填充缺失值为0
    bid_quantity_1[np.isnan(bid_quantity_1)] = 0
    bid_quantity_2[np.isnan(bid_quantity_2)] = 0
    bid_quantity_3[np.isnan(bid_quantity_3)] = 0
    ask_quantity_1[np.isnan(ask_quantity_1)] = 0
    ask_quantity_2[np.isnan(ask_quantity_2)] = 0
    ask_quantity_3[np.isnan(ask_quantity_3)] = 0
    
    return timestamp,order,bid_price_1,bid_price_2,bid_price_3,bid_quantity_1,\
            bid_quantity_2,bid_quantity_3,ask_price_1,ask_price_2,ask_price_3,ask_quantity_1,\
            ask_quantity_2,ask_quantity_3

In [20]:
# time_second是将timestamp数据格式转换为全按分钟计算
# time_second_basic是讲上面的转换结果减去一个32400
# 作用是使得上午的交易时间[0, 9000]对应09:00 ~ 11:30，下午的交易时间[14400, 25200]对应13:00 ~ 16:00，借以将上下午分开
def time_transform(timestamp_time):
    time_second_basic = []
    time_second = []
    for i in range(0,len(timestamp_time),1):
        second = float(timestamp_time[i][11])*36000 + float(timestamp_time[i][12])*3600+\
                    float(timestamp_time[i][14])*600 + float(timestamp_time[i][15])*60+\
                    float(timestamp_time[i][17])*10 + float(timestamp_time[i][18])  
        time_second_basic.append(second - 32400.0)
        time_second.append(second)
    return np.array(time_second),np.array(time_second_basic)

In [21]:
# rise factor
def rise_ask(Ask1,timestamp_time_second,before_time):
    Ask1[Ask1 == 0] = np.mean(Ask1)
    rise_ratio = []
    index = np.where(timestamp_time_second >= before_time)[0][0]
    #open first before_time mins
    for i in range(0, index, 1):
        rise_ratio_ = round((Ask1[i] - Ask1[0])*(1.0)/Ask1[0]*100,5)
        rise_ratio.append(rise_ratio_)
    for i in range(index,len(Ask1),1):
        #print np.where(timestamp_time_second[:i] >= timestamp_time_second[i] - before_time)
        #print timestamp_time_second[i],timestamp_time_second[i] - before_time
        index_start = np.where(timestamp_time_second[:i] >= timestamp_time_second[i] - before_time)[0][0]
        rise_ratio_ = round((Ask1[i] - Ask1[index_start])*(1.0)/Ask1[index_start]*100,5)
        rise_ratio.append(rise_ratio_)
    return np.array(rise_ratio)

In [22]:
# level-3 OBI and Depth Ratio
def weight_pecentage(w1,w2,w3,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3):
    Weight_Ask = (w1 * ask_quantity_1 + w2 * ask_quantity_2 + w3 * ask_quantity_3)
    Weight_Bid = (w1 * bid_quantity_1 + w2 * bid_quantity_2 + w3 * bid_quantity_3)

    W_AB = Weight_Ask/Weight_Bid
    W_A_B = (Weight_Ask - Weight_Bid)/(Weight_Ask + Weight_Bid)
    return W_AB, W_A_B

In [23]:
# 定义每个second的要predict的label，以及对应second级别的各类输入factor/feature，由于是second级别，输出的dataframe会比原来小
# 可以根据需要更改成tick级别的label和feature数据，这样输出的dataframe和原来大小差不多
def traded_label_one_second(time1,time2,time_second_basic,bid_price_1,ask_price_1, bid_quantity_1, ask_quantity_1, traded_time,\
                            rise_ratio_ask_1,rise_ratio_ask_2,rise_ratio_ask_3,rise_ratio_ask_4,\
                            rise_ratio_ask_5,rise_ratio_ask_6,rise_ratio_ask_7,rise_ratio_ask_8,\
                            rise_ratio_ask_9,rise_ratio_ask_10,rise_ratio_ask_11,rise_ratio_ask_12,\
                            rise_ratio_ask_13,rise_ratio_ask_14,rise_ratio_ask_15,rise_ratio_ask_16,\
                            rise_ratio_ask_17,rise_ratio_ask_18,rise_ratio_ask_19,rise_ratio_ask_20,\
                            rise_ratio_ask_21,rise_ratio_ask_22,rise_ratio_ask_23,rise_ratio_ask_24,\
                            rise_ratio_ask_25,rise_ratio_ask_26,rise_ratio_ask_27,rise_ratio_ask_28,\
                            rise_ratio_ask_29,rise_ratio_ask_30,W_AB_100, W_A_B_100, W_AB_010, W_A_B_010,\
                            W_AB_001,W_A_B_001, W_AB_910, W_A_B_910, W_AB_820, W_A_B_820, W_AB_730,\
                            W_A_B_730,W_AB_640, W_A_B_640, W_AB_550, W_A_B_550,W_AB_721, W_A_B_721,\
                            W_AB_532,W_A_B_532, W_AB_111, W_A_B_111, W_AB_190, W_A_B_190, W_AB_280,\
                            W_A_B_280,W_AB_370, W_A_B_370, W_AB_460, W_A_B_460, W_AB_127, W_A_B_127,\
                            W_AB_235, W_A_B_235):
    global index
    
    traded = []
    index_ = []
    
    rise_ratio_second_1 = []
    rise_ratio_second_2 = []   
    rise_ratio_second_3 = []
    rise_ratio_second_4 = []
    rise_ratio_second_5 = []
    rise_ratio_second_6 = []
    rise_ratio_second_7 = []
    rise_ratio_second_8 = []
    rise_ratio_second_9 = []
    rise_ratio_second_10 = []
    rise_ratio_second_11 = []
    rise_ratio_second_12 = []
    rise_ratio_second_13 = []
    rise_ratio_second_14 = []
    rise_ratio_second_15 = []
    rise_ratio_second_16 = []
    rise_ratio_second_17 = []
    rise_ratio_second_18 = []
    rise_ratio_second_19 = []
    rise_ratio_second_20 = []
    rise_ratio_second_21 = []
    rise_ratio_second_22 = []
    rise_ratio_second_23 = []
    rise_ratio_second_24 = []
    rise_ratio_second_25 = []
    rise_ratio_second_26 = []
    rise_ratio_second_27 = []
    rise_ratio_second_28 = []
    rise_ratio_second_29 = []
    rise_ratio_second_30 = []

    w_divid_100 = []
    w_diff_100 = []
    w_divid_010 = []
    w_diff_010 = []
    w_divid_001 = []
    w_diff_001 = []
    w_divid_910 = []
    w_diff_910 = []
    w_divid_820 = []
    w_diff_820 = []
    w_divid_730 = []
    w_diff_730 = []
    w_divid_640 = []
    w_diff_640 = []
    w_divid_550 = []
    w_diff_550 = []
    w_divid_721 = []
    w_diff_721 = []
    w_divid_532 = []
    w_diff_532 = []
    w_divid_111 = []
    w_diff_111 = []
    w_divid_190 = []
    w_diff_190 = []
    w_divid_280 = []
    w_diff_280 = []
    w_divid_370 = []
    w_diff_370 = []
    w_divid_460 = []
    w_diff_460 = []
    w_divid_127 = []
    w_diff_127 = []
    w_divid_235 = []
    w_diff_235 = []
    spread = []
    ask = []
    bid = []
    # micsecond = time_second_basic[np.where((time_second_basic >= time1) & (time_second_basic <= time2))]
    # for i in range(0,len(micsecond),1):
        
        # index = np.where(time_second_basic <= micsecond[i])[0][-1]
        # if i == 0:
        #     index_.append(index)
        # if i == len(micsecond) - 1:
        #     index_.append(index)
        # if micsecond[i] <= 25200 - traded_time:
        #     index_min = np.where(time_second_basic <= micsecond[i] + traded_time)[0][-1]
        #     if bid_price_1[index] > min(ask_price_1[index:index_min]):
        #         traded.append(1)
        #     else:
        #         traded.append(0)
        # elif micsecond[i] > 25200 - traded_time:
        #     if bid_price_1[index] > ask_price_1[-1]:
        #         traded.append(1)
        #     else:
        #         traded.append(0)

    if time1 == 0:
        index_one = np.where(time_second_basic <= 0)[0][-1]
    elif time1 == 14400:
        index_one = np.where(time_second_basic <= 14400)[0][-1]
        
    for i in range(time1, time2, 1):
        if i == 0 or i == 14400:
            index_array = np.where(time_second_basic <= i)[-1]
        else:
            index_array = np.where((time_second_basic < i+1) & (time_second_basic >= i))[-1]
        
        if len(index_array) > 0:
            index = index_array[-1]
            if i == time1:
                index_.append(index)
            if i == time2 - 1:
                index_.append(index)
            if i < 25200 - traded_time:
                index_min = np.where(time_second_basic <= i + traded_time)[0][-1]
                traded_min = ask_price_1[index:index_min]
                if bid_price_1[index] > min(traded_min):
                    traded.append(1)
                else:
                    traded.append(0)
            elif i >= 25200 - traded_time:
                if bid_price_1[index] > ask_price_1[-1]:
                    traded.append(1)
                else:
                    traded.append(0)
            ask0 = ask_price_1[index]
            bid0 = bid_price_1[index]
            ask.append(ask0)
            bid.append(bid0)
            spread.append(10000*(ask0-bid0)*2/(ask0+bid0))
            rise_ratio_second_1.append(rise_ratio_ask_1[(index - index_one)])
            rise_ratio_second_2.append(rise_ratio_ask_2[(index - index_one)])
            rise_ratio_second_3.append(rise_ratio_ask_3[(index - index_one)])
            rise_ratio_second_4.append(rise_ratio_ask_4[(index - index_one)])
            rise_ratio_second_5.append(rise_ratio_ask_5[(index - index_one)])
            rise_ratio_second_6.append(rise_ratio_ask_6[(index - index_one)])
            rise_ratio_second_7.append(rise_ratio_ask_7[(index - index_one)])
            rise_ratio_second_8.append(rise_ratio_ask_8[(index - index_one)])
            rise_ratio_second_9.append(rise_ratio_ask_9[(index - index_one)])
            rise_ratio_second_10.append(rise_ratio_ask_10[(index - index_one)])
            rise_ratio_second_11.append(rise_ratio_ask_11[(index - index_one)])
            rise_ratio_second_12.append(rise_ratio_ask_12[(index - index_one)])
            rise_ratio_second_13.append(rise_ratio_ask_13[(index - index_one)])
            rise_ratio_second_14.append(rise_ratio_ask_14[(index - index_one)])
            rise_ratio_second_15.append(rise_ratio_ask_15[(index - index_one)])
            rise_ratio_second_16.append(rise_ratio_ask_16[(index - index_one)])
            rise_ratio_second_17.append(rise_ratio_ask_17[(index - index_one)])
            rise_ratio_second_18.append(rise_ratio_ask_18[(index - index_one)])
            rise_ratio_second_19.append(rise_ratio_ask_19[(index - index_one)])
            rise_ratio_second_20.append(rise_ratio_ask_20[(index - index_one)])
            rise_ratio_second_21.append(rise_ratio_ask_21[(index - index_one)])
            rise_ratio_second_22.append(rise_ratio_ask_22[(index - index_one)])
            rise_ratio_second_23.append(rise_ratio_ask_23[(index - index_one)])
            rise_ratio_second_24.append(rise_ratio_ask_24[(index - index_one)])
            rise_ratio_second_25.append(rise_ratio_ask_25[(index - index_one)])
            rise_ratio_second_26.append(rise_ratio_ask_26[(index - index_one)])
            rise_ratio_second_27.append(rise_ratio_ask_27[(index - index_one)])
            rise_ratio_second_28.append(rise_ratio_ask_28[(index - index_one)])
            rise_ratio_second_29.append(rise_ratio_ask_29[(index - index_one)])
            rise_ratio_second_30.append(rise_ratio_ask_30[(index - index_one)])
            
            w_divid_100.append(W_AB_100[index_one + (index - index_one)])
            w_diff_100.append(W_A_B_100[index_one + (index - index_one)])
            w_divid_010.append(W_AB_010[index_one + (index - index_one)])
            w_diff_010.append(W_A_B_010[index_one + (index - index_one)])
            w_divid_001.append(W_AB_001[index_one + (index - index_one)])
            w_diff_001.append(W_A_B_001[index_one + (index - index_one)])
            w_divid_910.append(W_AB_910[index_one + (index - index_one)])
            w_diff_910.append(W_A_B_910[index_one + (index - index_one)])
            w_divid_820.append(W_AB_820[index_one + (index - index_one)])
            w_diff_820.append(W_A_B_820[index_one + (index - index_one)])
            w_divid_730.append(W_AB_730[index_one + (index - index_one)])
            w_diff_730.append(W_A_B_730[index_one + (index - index_one)])
            w_divid_640.append(W_AB_640[index_one + (index - index_one)])
            w_diff_640.append(W_A_B_640[index_one + (index - index_one)])
            w_divid_550.append(W_AB_550[index_one + (index - index_one)])
            w_diff_550.append(W_A_B_550[index_one + (index - index_one)])
            w_divid_721.append(W_AB_721[index_one + (index - index_one)])
            w_diff_721.append(W_A_B_721[index_one + (index - index_one)])
            w_divid_532.append(W_AB_532[index_one + (index - index_one)])
            w_diff_532.append(W_A_B_532[index_one + (index - index_one)])
            w_divid_111.append(W_AB_111[index_one + (index - index_one)])
            w_diff_111.append(W_A_B_111[index_one + (index - index_one)])
            w_divid_190.append(W_AB_190[index_one + (index - index_one)])
            w_diff_190.append(W_A_B_190[index_one + (index - index_one)])
            w_divid_280.append(W_AB_280[index_one + (index - index_one)])
            w_diff_280.append(W_A_B_280[index_one + (index - index_one)])
            w_divid_370.append(W_AB_370[index_one + (index - index_one)])
            w_diff_370.append(W_A_B_370[index_one + (index - index_one)])
            w_divid_460.append(W_AB_460[index_one + (index - index_one)])
            w_diff_460.append(W_A_B_460[index_one + (index - index_one)])
            w_divid_127.append(W_AB_127[index_one + (index - index_one)])
            w_diff_127.append(W_A_B_127[index_one + (index - index_one)])
            w_divid_235.append(W_AB_235[index_one + (index - index_one)])
            w_diff_235.append(W_A_B_235[index_one + (index - index_one)])
        
        elif len(index_array) == 0:
            if i < 25200 - traded_time:
                index_min = np.where(time_second_basic <= i + traded_time)[0][-1]
                traded_min = ask_price_1[index:index_min]
                if bid_price_1[index] > min(traded_min):
                    traded.append(1)
                else:
                    traded.append(0)
            elif i >= 25200 - traded_time:
                if bid_price_1[index] > ask_price_1[-1]:
                    traded.append(1)
                else:
                    traded.append(0)

            ask0 = ask_price_1[index]
            bid0 = bid_price_1[index]
            ask.append(ask0)
            bid.append(bid0)
            spread.append(10000*(ask0-bid0)*2/(ask0+bid0))
            rise_ratio_second_1.append(rise_ratio_second_1[-1])
            rise_ratio_second_2.append(rise_ratio_second_2[-1])
            rise_ratio_second_3.append(rise_ratio_second_3[-1])
            rise_ratio_second_4.append(rise_ratio_second_4[-1])
            rise_ratio_second_5.append(rise_ratio_second_5[-1])
            rise_ratio_second_6.append(rise_ratio_second_6[-1])
            rise_ratio_second_7.append(rise_ratio_second_7[-1])
            rise_ratio_second_8.append(rise_ratio_second_8[-1])
            rise_ratio_second_9.append(rise_ratio_second_9[-1])
            rise_ratio_second_10.append(rise_ratio_second_10[-1])
            rise_ratio_second_11.append(rise_ratio_second_11[-1])
            rise_ratio_second_12.append(rise_ratio_second_12[-1])
            rise_ratio_second_13.append(rise_ratio_second_13[-1])
            rise_ratio_second_14.append(rise_ratio_second_14[-1])
            rise_ratio_second_15.append(rise_ratio_second_15[-1])
            rise_ratio_second_16.append(rise_ratio_second_16[-1])
            rise_ratio_second_17.append(rise_ratio_second_17[-1])
            rise_ratio_second_18.append(rise_ratio_second_18[-1])
            rise_ratio_second_19.append(rise_ratio_second_19[-1])
            rise_ratio_second_20.append(rise_ratio_second_20[-1])
            rise_ratio_second_21.append(rise_ratio_second_21[-1])
            rise_ratio_second_22.append(rise_ratio_second_22[-1])
            rise_ratio_second_23.append(rise_ratio_second_23[-1])
            rise_ratio_second_24.append(rise_ratio_second_24[-1])
            rise_ratio_second_25.append(rise_ratio_second_25[-1])
            rise_ratio_second_26.append(rise_ratio_second_26[-1])
            rise_ratio_second_27.append(rise_ratio_second_27[-1])
            rise_ratio_second_28.append(rise_ratio_second_28[-1])
            rise_ratio_second_29.append(rise_ratio_second_29[-1])
            rise_ratio_second_30.append(rise_ratio_second_30[-1])
            
            w_divid_100.append(w_divid_100[-1])
            w_diff_100.append(w_diff_100[-1])
            w_divid_010.append(w_divid_010[-1])
            w_diff_010.append(w_diff_010[-1])
            w_divid_001.append(w_divid_001[-1])
            w_diff_001.append(w_diff_001[-1])
            w_divid_910.append(w_divid_910[-1])
            w_diff_910.append(w_diff_910[-1])
            w_divid_820.append(w_divid_820[-1])
            w_diff_820.append(w_diff_820[-1])
            w_divid_730.append(w_divid_730[-1])
            w_diff_730.append(w_diff_730[-1])
            w_divid_640.append(w_divid_640[-1])
            w_diff_640.append(w_diff_640[-1])
            w_divid_550.append(w_divid_550[-1])
            w_diff_550.append(w_diff_550[-1])
            w_divid_721.append(w_divid_721[-1])
            w_diff_721.append(w_diff_721[-1])
            w_divid_532.append(w_divid_532[-1])
            w_diff_532.append(w_diff_532[-1])
            w_divid_111.append(w_divid_111[-1])
            w_diff_111.append(w_diff_111[-1])
            w_divid_190.append(w_divid_190[-1])
            w_diff_190.append(w_diff_190[-1])
            w_divid_280.append(w_divid_280[-1])
            w_diff_280.append(w_diff_280[-1])
            w_divid_370.append(w_divid_370[-1])
            w_diff_370.append(w_diff_370[-1])
            w_divid_460.append(w_divid_460[-1])
            w_diff_460.append(w_diff_460[-1])
            w_divid_127.append(w_divid_127[-1])
            w_diff_127.append(w_diff_127[-1])
            w_divid_235.append(w_divid_235[-1])
            w_diff_235.append(w_diff_235[-1])
            
    return traded,index_,rise_ratio_second_1,rise_ratio_second_2,rise_ratio_second_3,\
            rise_ratio_second_4,rise_ratio_second_5,rise_ratio_second_6,rise_ratio_second_7,\
            rise_ratio_second_8,rise_ratio_second_9,rise_ratio_second_10,rise_ratio_second_11,\
            rise_ratio_second_12,rise_ratio_second_13,rise_ratio_second_14,rise_ratio_second_15,\
            rise_ratio_second_16,rise_ratio_second_17,rise_ratio_second_18,rise_ratio_second_19,\
            rise_ratio_second_20,rise_ratio_second_21,rise_ratio_second_22,rise_ratio_second_23,\
            rise_ratio_second_24,rise_ratio_second_25,rise_ratio_second_26,rise_ratio_second_27,\
            rise_ratio_second_28,rise_ratio_second_29,rise_ratio_second_30,w_divid_100,w_diff_100,\
            w_divid_010,w_diff_010,w_divid_001,w_diff_001,w_divid_910,w_diff_910,w_divid_820,w_diff_820,\
            w_divid_730,w_diff_730,w_divid_640,w_diff_640,w_divid_550,w_diff_550,w_divid_721,w_diff_721,\
            w_divid_532,w_diff_532,w_divid_111,w_diff_111,w_divid_190,w_diff_190,w_divid_280,w_diff_280,\
            w_divid_370,w_diff_370,w_divid_460,w_diff_460,w_divid_127,w_diff_127,w_divid_235,w_diff_235, spread, ask, bid

In [24]:
# 根据输入时间，分别得到上下午的因子值

# # 上午：只取09:00 ~ 11:30作为交易区间，对应：
# time1 = 0
# time2 = 9000

# # 下午：只取13:00 ~ 16:00作为交易区间，对应：
# time1 = 14400
# time2 = 25200


def get_feature(time1, time2, traded_time,time_second_basic,bid_price_1,ask_price_1, bid_quantity_1, ask_quantity_1, rise_ratio_ask_1,\
                         rise_ratio_ask_2,rise_ratio_ask_3,rise_ratio_ask_4,rise_ratio_ask_5,\
                         rise_ratio_ask_6,rise_ratio_ask_7,rise_ratio_ask_8,rise_ratio_ask_9,\
                         rise_ratio_ask_10,rise_ratio_ask_11,rise_ratio_ask_12,rise_ratio_ask_13,\
                         rise_ratio_ask_14,rise_ratio_ask_15,rise_ratio_ask_16,rise_ratio_ask_17,\
                         rise_ratio_ask_18,rise_ratio_ask_19,rise_ratio_ask_20,rise_ratio_ask_21,\
                         rise_ratio_ask_22,rise_ratio_ask_23,rise_ratio_ask_24,rise_ratio_ask_25,\
                         rise_ratio_ask_26,rise_ratio_ask_27,rise_ratio_ask_28,rise_ratio_ask_29,\
                         rise_ratio_ask_30,W_AB_100, W_A_B_100, W_AB_010, W_A_B_010, W_AB_001,\
                         W_A_B_001, W_AB_910, W_A_B_910, W_AB_820, W_A_B_820, W_AB_730 , W_A_B_730,\
                         W_AB_640, W_A_B_640, W_AB_550, W_A_B_550,W_AB_721, W_A_B_721, W_AB_532,\
                         W_A_B_532, W_AB_111, W_A_B_111, W_AB_190, W_A_B_190, W_AB_280 , W_A_B_280,\
                         W_AB_370, W_A_B_370, W_AB_460, W_A_B_460, W_AB_127, W_A_B_127, W_AB_235, W_A_B_235):

    traded,index_,rise_ratio_second_1,rise_ratio_second_2,rise_ratio_second_3,\
    rise_ratio_second_4,rise_ratio_second_5,rise_ratio_second_6,rise_ratio_second_7,\
    rise_ratio_second_8,rise_ratio_second_9,rise_ratio_second_10,rise_ratio_second_11,\
    rise_ratio_second_12,rise_ratio_second_13,rise_ratio_second_14,rise_ratio_second_15,\
    rise_ratio_second_16,rise_ratio_second_17,rise_ratio_second_18,rise_ratio_second_19,\
    rise_ratio_second_20,rise_ratio_second_21,rise_ratio_second_22,rise_ratio_second_23,\
    rise_ratio_second_24,rise_ratio_second_25,rise_ratio_second_26,rise_ratio_second_27,\
    rise_ratio_second_28,rise_ratio_second_29,rise_ratio_second_30,w_divid_100,w_diff_100,\
    w_divid_010,w_diff_010,w_divid_001,w_diff_001,w_divid_910,w_diff_910,w_divid_820,w_diff_820,\
    w_divid_730,w_diff_730,w_divid_640,w_diff_640,w_divid_550,w_diff_550,w_divid_721,w_diff_721,\
    w_divid_532,w_diff_532,w_divid_111,w_diff_111,w_divid_190,w_diff_190,w_divid_280,w_diff_280,\
    w_divid_370,w_diff_370,w_divid_460,w_diff_460,w_divid_127,w_diff_127,w_divid_235,w_diff_235, spread, ask, bid=\
        traded_label_one_second(time1,time2,time_second_basic,bid_price_1,ask_price_1, bid_quantity_1, ask_quantity_1, traded_time,\
                                rise_ratio_ask_1,rise_ratio_ask_2,rise_ratio_ask_3,rise_ratio_ask_4,\
                                rise_ratio_ask_5,rise_ratio_ask_6,rise_ratio_ask_7,rise_ratio_ask_8,\
                                rise_ratio_ask_9,rise_ratio_ask_10,rise_ratio_ask_11,rise_ratio_ask_12,\
                                rise_ratio_ask_13,rise_ratio_ask_14,rise_ratio_ask_15,rise_ratio_ask_16,\
                                rise_ratio_ask_17,rise_ratio_ask_18,rise_ratio_ask_19,rise_ratio_ask_20,\
                                rise_ratio_ask_21,rise_ratio_ask_22,rise_ratio_ask_23,rise_ratio_ask_24,\
                                rise_ratio_ask_25,rise_ratio_ask_26,rise_ratio_ask_27,rise_ratio_ask_28,\
                                rise_ratio_ask_29,rise_ratio_ask_30,W_AB_100, W_A_B_100, W_AB_010, W_A_B_010,\
                                W_AB_001,W_A_B_001, W_AB_910, W_A_B_910, W_AB_820, W_A_B_820, W_AB_730,\
                                W_A_B_730,W_AB_640, W_A_B_640, W_AB_550, W_A_B_550,W_AB_721, W_A_B_721,\
                                W_AB_532,W_A_B_532, W_AB_111, W_A_B_111, W_AB_190, W_A_B_190, W_AB_280,\
                                W_A_B_280,W_AB_370, W_A_B_370, W_AB_460, W_A_B_460, W_AB_127, W_A_B_127,\
                                W_AB_235, W_A_B_235)
    
    data = np.array([traded,rise_ratio_second_1,rise_ratio_second_2,rise_ratio_second_3,\
                    rise_ratio_second_4,rise_ratio_second_5,rise_ratio_second_6,rise_ratio_second_7,\
                    rise_ratio_second_8,rise_ratio_second_9,rise_ratio_second_10,rise_ratio_second_11,\
                    rise_ratio_second_12,rise_ratio_second_13,rise_ratio_second_14,rise_ratio_second_15,\
                    rise_ratio_second_16,rise_ratio_second_17,rise_ratio_second_18,rise_ratio_second_19,\
                    rise_ratio_second_20,rise_ratio_second_21,rise_ratio_second_22,rise_ratio_second_23,\
                    rise_ratio_second_24,rise_ratio_second_25,rise_ratio_second_26,rise_ratio_second_27,\
                    rise_ratio_second_28,rise_ratio_second_29,rise_ratio_second_30,w_divid_100,w_diff_100,\
                    w_divid_010,w_diff_010,w_divid_001,w_diff_001,w_divid_910,w_diff_910,w_divid_820,w_diff_820,\
                    w_divid_730,w_diff_730,w_divid_640,w_diff_640,w_divid_550,w_diff_550,w_divid_721,w_diff_721,\
                    w_divid_532,w_diff_532,w_divid_111,w_diff_111,w_divid_190,w_diff_190,w_divid_280,w_diff_280,\
                    w_divid_370,w_diff_370,w_divid_460,w_diff_460,w_divid_127,w_diff_127,w_divid_235,w_diff_235, spread, ask, bid]).T

    return pd.DataFrame(data)#,traded_1 #, columns = ['label', 'rise', 'depth_divid', 'depth_diff'])


In [27]:
def data(month,day,traded_time):
    
    # 读取所有bid1-3, ask1-3, bidqty1-3, askqty1-3，订单时间的序列
    # order_book_此处没有用到，是读取的dataframe本身
    timestamp, order_book_ ,bid_price_1, bid_price_2, bid_price_3, bid_quantity_1, bid_quantity_2, bid_quantity_3, ask_price_1, ask_price_2, ask_price_3, ask_quantity_1, ask_quantity_2, ask_quantity_3 = order_book(month,day)
    

    # 得到计算的时间戳
    time_second, time_second_basic = time_transform(timestamp)
    # Ask1是上午9:00之后卖一价
    Ask1 = ask_price_1[np.where(time_second_basic <= 0.0)[0][-1]:]

    # 设置不同权重创建一系列rise ratio factor作为输入feature
    before_time = 60.0 * 6
    rise_ratio_ask_1 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 6 + 30 
    rise_ratio_ask_2 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 7
    rise_ratio_ask_3 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 7 + 30 
    rise_ratio_ask_4 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 8
    rise_ratio_ask_5 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 8 + 30
    rise_ratio_ask_6 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 9
    rise_ratio_ask_7 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 9 + 30
    rise_ratio_ask_8 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 10
    rise_ratio_ask_9 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 10 + 30
    rise_ratio_ask_10 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 11
    rise_ratio_ask_11 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 11 + 30
    rise_ratio_ask_12 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 12
    rise_ratio_ask_13 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 12 + 30
    rise_ratio_ask_14 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 13
    rise_ratio_ask_15 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 13 + 30
    rise_ratio_ask_16 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 14
    rise_ratio_ask_17 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 14 + 30 
    rise_ratio_ask_18 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 15
    rise_ratio_ask_19 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 15 + 30
    rise_ratio_ask_20 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 16
    rise_ratio_ask_21 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 16 + 30
    rise_ratio_ask_22 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 17
    rise_ratio_ask_23 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 17 + 30
    rise_ratio_ask_24 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 18
    rise_ratio_ask_25 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 18 + 30
    rise_ratio_ask_26 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 19
    rise_ratio_ask_27 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 19 + 30
    rise_ratio_ask_28 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 20 
    rise_ratio_ask_29 = rise_ask(Ask1, time_second_basic, before_time)
    before_time = 60.0 * 20 + 30
    rise_ratio_ask_30 = rise_ask(Ask1, time_second_basic, before_time)
    
    # 设置不同权重创建一系列depth ratio factor、OBI signal作为输入feature
    W_AB_100 , W_A_B_100 = weight_pecentage(100, 0.0, 0.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_010 , W_A_B_010 = weight_pecentage(0.0, 100.0, 0.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_001 , W_A_B_001 = weight_pecentage(0.0, 0.0, 100.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_910 , W_A_B_910 = weight_pecentage(90.0, 10.0, 0.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_820 , W_A_B_820 = weight_pecentage(80.0, 20.0, 0.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_730 , W_A_B_730 = weight_pecentage(70.0, 30.0, 0.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_640 , W_A_B_640 = weight_pecentage(60.0, 40.0, 0.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_550 , W_A_B_550 = weight_pecentage(50.0, 50.0, 0.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_721 , W_A_B_721 = weight_pecentage(70.0, 20.0, 10.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_532 , W_A_B_532 = weight_pecentage(50.0, 30.0, 20.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_111 , W_A_B_111 = weight_pecentage(1.0, 1.0, 1.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_190 , W_A_B_190 = weight_pecentage(10.0, 90.0, 1.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_280 , W_A_B_280 = weight_pecentage(20.0, 80.0, 0.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_370 , W_A_B_370 = weight_pecentage(30.0, 70.0, 0.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_460 , W_A_B_460 = weight_pecentage(40.0, 60.0, 0.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_127 , W_A_B_127 = weight_pecentage(10.0, 20.0, 70.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    W_AB_235 , W_A_B_235 = weight_pecentage(20.0, 30.0, 50.0,ask_quantity_1,ask_quantity_2,ask_quantity_3, bid_quantity_1,bid_quantity_2,bid_quantity_3)
    
    # 上午：只取09:00 ~ 11:30作为交易区间，对应：
    time1 = 0
    time2 = 9000

    # 分别计算交易日上午的输入feature
    data_2014_UP =\
    get_feature(time1, time2, traded_time,time_second_basic,bid_price_1,ask_price_1, bid_quantity_1, ask_quantity_1, rise_ratio_ask_1,\
                         rise_ratio_ask_2,rise_ratio_ask_3,rise_ratio_ask_4,rise_ratio_ask_5,\
                         rise_ratio_ask_6,rise_ratio_ask_7,rise_ratio_ask_8,rise_ratio_ask_9,\
                         rise_ratio_ask_10,rise_ratio_ask_11,rise_ratio_ask_12,rise_ratio_ask_13,\
                         rise_ratio_ask_14,rise_ratio_ask_15,rise_ratio_ask_16,rise_ratio_ask_17,\
                         rise_ratio_ask_18,rise_ratio_ask_19,rise_ratio_ask_20,rise_ratio_ask_21,\
                         rise_ratio_ask_22,rise_ratio_ask_23,rise_ratio_ask_24,rise_ratio_ask_25,\
                         rise_ratio_ask_26,rise_ratio_ask_27,rise_ratio_ask_28,rise_ratio_ask_29,\
                         rise_ratio_ask_30,W_AB_100, W_A_B_100, W_AB_010, W_A_B_010, W_AB_001,\
                         W_A_B_001, W_AB_910, W_A_B_910, W_AB_820, W_A_B_820, W_AB_730 , W_A_B_730,\
                         W_AB_640, W_A_B_640, W_AB_550, W_A_B_550,W_AB_721, W_A_B_721, W_AB_532,\
                         W_A_B_532, W_AB_111, W_A_B_111, W_AB_190, W_A_B_190, W_AB_280 , W_A_B_280,\
                         W_AB_370, W_A_B_370, W_AB_460, W_A_B_460, W_AB_127, W_A_B_127, W_AB_235, W_A_B_235)

    # 下午：只取13:00 ~ 16:00作为交易区间，对应：
    time1 = 14400
    time2 = 25200

    # 计算交易日下午的输入feature
    data_2014_DOWN =\
    get_feature(time1, time2, traded_time,time_second_basic,bid_price_1,ask_price_1, bid_quantity_1, ask_quantity_1, rise_ratio_ask_1,\
                           rise_ratio_ask_2,rise_ratio_ask_3,rise_ratio_ask_4,rise_ratio_ask_5,\
                           rise_ratio_ask_6,rise_ratio_ask_7,rise_ratio_ask_8,rise_ratio_ask_9,\
                           rise_ratio_ask_10,rise_ratio_ask_11,rise_ratio_ask_12,rise_ratio_ask_13,\
                           rise_ratio_ask_14,rise_ratio_ask_15,rise_ratio_ask_16,rise_ratio_ask_17,\
                           rise_ratio_ask_18,rise_ratio_ask_19,rise_ratio_ask_20,rise_ratio_ask_21,\
                           rise_ratio_ask_22,rise_ratio_ask_23,rise_ratio_ask_24,rise_ratio_ask_25,\
                           rise_ratio_ask_26,rise_ratio_ask_27,rise_ratio_ask_28,rise_ratio_ask_29,\
                           rise_ratio_ask_30,W_AB_100, W_A_B_100, W_AB_010, W_A_B_010, W_AB_001,\
                           W_A_B_001, W_AB_910, W_A_B_910, W_AB_820, W_A_B_820, W_AB_730 , W_A_B_730,\
                           W_AB_640, W_A_B_640, W_AB_550, W_A_B_550,W_AB_721, W_A_B_721, W_AB_532,\
                           W_A_B_532, W_AB_111, W_A_B_111, W_AB_190, W_A_B_190, W_AB_280 , W_A_B_280,\
                           W_AB_370, W_A_B_370, W_AB_460, W_A_B_460, W_AB_127, W_A_B_127, W_AB_235, W_A_B_235)

    return data_2014_UP,data_2014_DOWN, len(W_AB_111) #trade_1, trade_2, timestamp

In [28]:
# 将上午和下午计算的因子分别存档
# traded_time为产生人工label的参数，一般取15min，对应300s，详见HFT_factor.ipynb

def train_test_to_csv(month,day,traded_time):
    data_UP,data_DOWN, len_ = data(month,day,traded_time)
    data_UP.to_csv('./processed_data/order_book_3_2014_new'+'_'+str(month)+'_'+str(day)+'_'+'UP'+'.csv', index = False)
    data_DOWN.to_csv('./processed_data/order_book_3_2014_new'+'_'+str(month)+'_'+str(day)+'_'+'DOWN'+'.csv', index = False)   

if __name__ == "__main__":
    # 以一天为例，有多天可以循环产生文件，后续model fit部分可以append起来
    month = 1
    day_ = [2]
    traded_time = 600
    for i in day_:
        print(i)
        train_test_to_csv(month,i,600) 

2
